# コンピューティングを操作する

Azure Machine Learning 実験としてスクリプトを実行する場合は、実験実行の実行コンテキストを定義する必要があります。実行コンテキストは以下で構成されます。

* スクリプト向けの Python 環境。スクリプトで使用するすべての Python パッケージを含める必要があります。
* スクリプトが実行されるコンピューティング ターゲット。実験実行を開始するローカル ワークステーション、またはオンデマンドで提供されるトレーニング クラスターなどのリモート コンピューター先になります。

このノートブックでは、実験の*環境*と*コンピューティング ターゲット*について学びます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用データを準備する

このノートブックでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (すでに存在する場合は、コードによって既存のバージョンを検索します)

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## トレーニング スクリプトを作成する

次の 2 つのセルを実行して作成します。

1. 新しい実験用のフォルダー
2. **SCIkit-learn を使用** してモデルをトレーニングし、**matplotlib** を使用して ROC 曲線をプロットするトレーニング スクリプト ファイル。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 環境を定義する

Azure Machine Learning で実験として Python スクリプトを実行すると、Conda 環境が作成され、スクリプトの実行コンテキストが定義されます。Azure Machine Learning には、多くの共通パッケージを含む既定の環境を提供します。これには、実験実行の操作に必要なライブラリを含む **azureml-defaults** パッケージ、**Pandas** や **numpy** などの一般なパッケージが含まれます。

また、**Conda** または **PIP** を使用して Conda 仕様ファイルで独自の環境を定義し、パッケージを追加して、実験が必要なすべてのライブラリにアクセスできるようにすることもできます。

> **注**: Conda 依存関係を最初にインストールした後、pip 依存関係をインストールします。**pip** パッケージは pip 依存関係をインストールするために必要になるため、これを conda 依存関係に含めることが推薦されます。

次のセルを実行して、このノートブックと同じフォルダーに *experiment_env.yml* という名前の Conda 仕様ファイルを作成します。

In [ ]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

これで、カスタム conda 仕様ファイルを使用して、実験用の環境を作成することが可能になります。

In [ ]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

これで、環境を使用し、スクリプトを実験として実行することができます。

次のコードでは、作成した環境を ScriptRunConfig に割り当て、実験を送信します。実験の実行中に、ウィジェットおよび **azureml_logs/60_control_log.txt** 出力ログで実行の詳細を確認すると、Conda 環境が構築されていることがわかります。

> **注**: 以下のコードは、スクリプト実行に使用する **DockerConfiguration** を作成し、スクリプトの環境を Docker コンテナーでホストするために、その **use_docker** 属性を **True** に設定します。これはデフォルトの動作であるため、省略して構いませんが、明示する目的でここに含めています。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

実験では、必要なすべてのパッケージを含む環境が正常に使用されました - Azure Machine Learning Studio で実行された実験のメトリックと出力を表示するか、以下のコード (**Scikit-learn** を使用してトレーニングされたモデルや **matplotlib を**使用して生成された ROC チャート イメージを含む) を実行して表示できます。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 環境を登録する

必要なパッケージを使用して環境を定義する手間が省けたので、ワークスペースに登録できます。

In [ ]:
# Register the environment
experiment_env.register(workspace=ws)

環境は、最初に作成したときに割り当てた名前 (この場合、*diabetes-experiment-env*) で登録されていることに注意してください。

環境が登録されている場合、同じ要件を持つすべてのスクリプトに再利用できます。たとえば、別のアルゴリズムを使用して糖尿病モデルをトレーニングするフォルダーとスクリプトを作成してみましょう。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、登録された環境を取得し、代替トレーニング スクリプトを実行する新しい実験でこれを使用できます (デシジョン ツリー分類子は正規化パラメーターを必要としないため、ここでは使われていません)。

In [ ]:
# get the registered environment
registered_env = Environment.get(ws, 'experiment_env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=registered_env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

今回は、一致する環境が前回の実行からキャッシュされており、ローカル コンピューティングで再作成する必要がないため、実験の実行速度が速くなります。ただし、異なるコンピューティング ターゲットでも、同じ環境が作成および使用され、実験スクリプトの実行コンテキストの一貫性が確保されます。

実験のメトリックと出力を見てみましょう。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 登録済み環境を表示する

独自の環境を登録するだけでなく、一般的な実験タイプに対して事前構築され「選別された」環境を活用できます。次のコードは、登録されているすべての環境を一覧表示します。

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

すべての選別された環境には、***AzureML-*** で始まる名前が付いています (このプレフィックスは、独自の環境では使用できません)。

## コンピューティング クラスターを作成する

多くの場合、ローカル コンピューティングリソースでは、大量のデータを処理する必要がある複雑な実験や長時間実行される実験を処理するには十分でない場合があります。また、クラウドでコンピューティング リソースを動的に作成して使用する機能を活用する場合もあります。Azure Machine Learning は、さまざまなコンピューティング ターゲットをサポートしており、これをワークスペースで定義し、実験の実行に使用できます。リソースの支払いは使用時にのみ行われます。

コンピューティング クラスターは、[Azure Machine Learning Studio](https://ml.azure.com) で作成するか、Azure Machine Learning SDK を使用して作成できます。以下のコード セルは指定された名前を使ってコンピューティング クラスターがあるかどうかワークスペースを確認し、ない場合は作成します。

> **重要**: 実行する前に、以下のコードで *your-compute-cluster* をコンピューティング クラスターに適した名前に変更してください。既存のクラスターがある場合はその名前を指定できます。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

> **注**: コンピューティング インスタンスとクラスターは、スタンダードの Azure 仮想マシンのイメージに基づいています。この演習では、コストとパフォーマンスの最適なバランスを実現するために、*Standard_DS11_v2* イメージが推薦されます。サブスクリプションにこのイメージを含まないクォータがある場合は、別のイメージを選択してください。 ただし、画像が大きいほどコストが高くなり、小さすぎるとタスクが完了できない場合があることに注意してください。Azure 管理者にクォータを拡張するように依頼していただくことも可能です。

## リモート コンピューティングで実験を実行する

これで、以前に実行した実験を再実行する準備が整いましたが、今回は作成したコンピューティング クラスターで実行します。 

> **注**: コンテナー イメージは Conda 環境で構築する必要があり、スクリプトを実行する前にクラスター ノードを起動してイメージをデプロイする必要があるため、実験にはかなり時間がかかります。糖尿病トレーニング スクリプトのような簡単な実験では、これは非効率的に見えるかもしれません。しかし、数時間かかるより複雑な実験を実行する必要があると想像してください - よりスケーラブルな計算を動的に作成すると、全体の時間が大幅に短縮される可能性があります。

In [ ]:
# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

実験の実行を待っている間に、上のウィジェットまたは [Azure Machine Learning Studio](https://ml.azure.com) でコンピューティングの状態を確認できます。次のコマンドを使用して、コンピューティングの状態を確認することもできます。

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

状態が*安定*から*サイズ変更中*に変わるまでにはしばらく時間がかかることに注意してください (コーヒーブレイクをするのによいタイミングです)。実行が完了するまでカーネルをブロックするには、下のセルを実行します。

In [ ]:
run.wait_for_completion()

ページの右上にあるカーネル インジケータに注目してください。**&#9899;** から **&#9711;** に変わると、コードの実行が終了します。

実験が完了したら、実験の実行によって生成されたメトリックとファイルを取得できます。今回は、ファイルには、イメージを構築し、コンピューティングを管理するためのログが含まれます。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

これで、実験によってトレーニングされたモデルを登録できます。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**:
>
> - Azure Machine Learning での環境の詳細については、[Azure Machine Learning でソフトウェア環境を作成して使用する](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)を参照してください。
> - Azure Machine Learning でのコンピューティング ターゲットの詳細については、[Azure Machine Learning のコンピューティング ターゲットとは](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)を参照してください。